In [33]:
from flask import Flask
from flask import Response
from flask import request
from flask_restful import Resource
from flask_restful import Api
from flaskext.mysql import MySQL
from functools import wraps

import json
import re
import sys

sys.path.append("../Common")

from modChatbot import modChatbot
from libmysql   import dbConMysql

#helper functions
def dictToJson(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        res = f(*args, **kwargs)
        res = json.dumps(res, ensure_ascii=False).encode('utf8')
        return Response(res, content_type='application/json; charset=utf-8')
    return decorated_function

#global variables 
app = Flask(__name__)
api = Api(app)    

In [34]:
@app.route("/")
def indexslash():
    return { 'data': u'파워프로그래머' }

@app.route("/index")
def index():
    return { 'data': u'파워프로그래머' }

@app.route("/db2json", methods=["GET","POST"])    
def db2json():                    
    config_bible = {
      'user': 'root',
      'password': 'karisma*3%7*4',
      'host': 'mthx.cafe24.com',
      'database': 'bible',
      'raise_on_warnings': True,
    }
    dbbible = dbConMysql( config_bible )
    query = "select * from TB_BIBLE"
    result = dbbible.selectQuery(query)
    print(result)        
    res = json.dumps(result, ensure_ascii=False).encode('utf8')
    return Response(res, content_type='application/json; charset=utf-8')

@app.route("/modtest", methods=["GET","POST"])    
@dictToJson
def modtest():                         
    print( modBot.isExistUsr( 'aaaa' ) )
    modBot.addUser( 'dddd' )            
    modBot.updateUserStat('U9A9agpChomm',0,0,0)        
    print("except")    

@app.route("/keyboard", methods=["GET","POST"])    
def keyboard() :
    print(request)
    return "{\"type\":\"text\"}"

def makeKakaoMsg( szMsg , listMenu ):
    print(szMsg, listMenu )
    if( len(listMenu) != 0 ):
        dicOutput = {"message":{"text":str(szMsg)},"keyboard":{"type":"buttons","buttons":listMenu}}        
    else :
        dicOutput = {"message":{"text":str(szMsg)}}
    return dicOutput

def parseSearch( msg ) :
    pats = ["([가-힣]*)([0-9]*):([0-9]*)(-)([0-9]*)" ,"([가-힣]*)([0-9]*):([0-9]*)(-)" ,"([가-힣]*)([0-9]*):([0-9]*)" ,"([가-힣]*)([0-9]*):" ,"([가-힣]*)([0-9]*)","([가-힣]*)"]
    listDesc = ["book","chap","no_start","isDash","no_end"]    
    dicRet = {}
    for pat in pats : 
        parser = re.compile( pat )
        m = parser.match(msg)
        if( m != None ) : 
            print( m.groups() , type(m.groups()))
            ret_tuple = m.groups()
            if( len(ret_tuple) ) : 
                dicRet['ret']='ok'
                for i in range(0,len(ret_tuple)) :
                    if( ret_tuple[i] != "" ) :                         
                        if( listDesc[i] in ['chap','no_start','no_end'] ) : dicRet[listDesc[i]] = int(ret_tuple[i])
                        else : dicRet[listDesc[i]] = ret_tuple[i]                    
    print( dicRet )            
    return dicRet
            
@app.route("/message", methods=["GET","POST"])    
@dictToJson
def message() :       
    if False : 
        dicJson = json.loads(request.data)
        print(dicJson['user_key'])
        print(dicJson['type'])        
        print(dicJson['content'])
        dicMessage = {"message":{"text":dicJson['content']}}
        return dicMessage
    else :        
        dicJson = json.loads(request.data)        
        usr = dicJson['user_key']
        media_type = dicJson['type']
        msg = dicJson['content']        
        reply,menu = replayFunction( usr, msg )
        ret = makeKakaoMsg(reply,menu)        
        return ret

In [35]:

def replayFunction(usr ,msg ):
    listBaseMenu = ['성경검색','성경구절검색','성경요절','성경QT','파파고번역K->E','파파고번역E->K']    
    listMenuMsg = ['기능','메뉴','처음으로','처음']
    
    if( modBot.isExistUsr( usr ) == False ) : 
        modBot.addUser( usr )        
        reply = "새로운 유저이신구요\n %s님 환영합니다\n사용하실 수 있는 기능은 다음과 같아요" % (usr)
        listMenu = listBaseMenu
    else :
        dicUsrStat = modBot.getUserStat(usr)[0];
        print( "dicUsrStat=",dicUsrStat)
                
        if dicUsrStat['statel1'] == 0 : #BaseMenu 선택중
            selIdx = 9999 #invalid object
            for i,menu in enumerate(listBaseMenu) :
                if menu == msg : 
                    selIdx = i
                    break
            if( selIdx != 9999 ) : 
                modBot.updateUserStat(usr,selIdx+1,0,0)
                reply = "%s님 %s를 선택하셨군요" % (usr,listBaseMenu[selIdx])
                listMenu = []                
            else: 
                modBot.updateUserStat(usr,0,0,0)
                reply ="%s님 사용하실 수 있는 기능은 다음과 같아요 ^^" % (usr)
                listMenu = listBaseMenu                
        else : 
            if( msg in listMenuMsg ) : 
                modBot.updateUserStat(usr,0,0,0)
                reply ="%s님 기능 메뉴를 다시 보여드릴게요 ^^" % (usr)
                listMenu = listBaseMenu                
            else : 
                # function별 
                if dicUsrStat['statel1'] == 1 : # 성경검색 
                    ret_parse = parseSearch (msg) 
                    ret = modBot.searchBible( ret_parse )
                    if( ret['ret'] == 'ok') :
                        reply ="%s님 선택하신 구절은 다음과 같아요\n%s" % (usr,ret['msg'])                        
                    else :
                        reply ="%s님 선택하신 구절은 유효하지 않습니다." % (usr)
                    listMenu = []                
                elif dicUsrStat['statel1'] == 2:
                    ret = modBot.searchBibleKey( msg )
                    if( ret['ret'] == 'ok') :
                        reply ="%s님 선택하신 키워드 검색 결과는 다음과 같아요\n%s" % (usr,ret['msg'])                        
                    else :
                        reply ="%s님 선택하신 키워드 검색 결과가 없습니다." % (usr)
                    listMenu = []                
                else :                    
                    modBot.updateUserStat(usr,0,0,0)
                    reply ="%s님 기능 메뉴를 다시 보여드릴게요 ^^" % (usr)
                    listMenu = listBaseMenu                                        
    return reply,listMenu

In [36]:
modBot = modChatbot()            

In [ ]:
if __name__ == '__main__':        
    if False :
        app.run(debug=True , host='0.0.0.0')
    else : #jupyer
        from werkzeug.serving import run_simple
        run_simple('0.0.0.0', 5000, app)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
[2018-07-19 16:32:12,426] ERROR in app: Exception on /index [GET]
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1974, in make_response
    rv = self.response_class.force_type(rv, request.environ)
  File "/root/anaconda3/lib/python3.6/site-packages/werkzeug/wrappers.py", line 921, in force_type
    response = BaseResponse(*_run_wsgi_app(response, environ))
  File "/root/anaconda3/lib/python3.6/site-packages/werkzeug/test.py", line 923, in run_wsgi_app
    app_rv = app(environ, start_response)
TypeError: 'dict' object is not callable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1816, in full_dispatch_request
 